## Архитектуры нейронок для рахных задач

In [ ]:
# 1. Координаты X, Y, TVD1 и TVD2 лежат отдельными листами в .xlsx файле
# 2. Файлов .tfrecord не пугаться, в деталях коммита на всякий указал как открывать такие. 
# Там 4 таких файла уже в порядке следования дат, для нефти, воды, пластового и забойного.
# Каждый тензор размерности [121,6,6]. 
# 3. Чтобы узнать какой дате соответствует индекс в тензоре, можно заюзать файл Dates_processed.csv.
# Загружаете как датафрейм и запрашиваете по индексу строки, получаете дату.

In [31]:
# imports 

import tensorflow as tf
import os 
import pandas as pd

In [ ]:
# specify your directory where files are located 
# tensors loading

files_dir = '../data/processed/'
loaded_tensors = dict()
for file in [x for x in os.listdir(files_dir) if x.endswith("tfrecord")]:
    serialized_tensor = tf.io.read_file(os.path.join(files_dir,file))
    loaded_tensors[file] = tf.io.parse_tensor(serialized_tensor, out_type=tf.float32)

# coordinates loading 
df_coords = pd.read_excel(os.path.join(files_dir,"Coordinates_processed.xlsx"),sheet_name=['X','Y','TVD1','TVD2'],header=None)
df_X_Y = pd.DataFrame({col: list(zip(df_coords["X"][col], df_coords["Y"][col])) for col in  df_coords["X"].columns})
df_TVD = pd.DataFrame({col: list(zip(df_coords["TVD1"][col], df_coords["TVD2"][col])) for col in  df_coords["TVD1"].columns})

# time loading 
time_sequence = pd.read_csv(os.path.join(files_dir,"Dates_processed.csv"))

### Архитектура для предсказания дебита нефти - input 1